**참고 자료**
- https://huggingface.co/docs/transformers/tasks/multiple_choice
- https://www.kaggle.com/code/bennyfung/bert-model-for-multiple-choice

### **1. 세팅**

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, Trainer
from datasets import Dataset
import re
import torch
import json
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# 데이터 로딩 함수
def load_data(file_path):
   return pd.read_json(file_path, encoding='utf-8-sig')

# 데이터 로딩
train = load_data('resource/data/대화맥락추론_train.json')
dev = load_data('resource/data/대화맥락추론_dev.json')
test = load_data('resource/data/대화맥락추론_test.json')

/data/jysuh/env/blossom/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-12 17:52:14.258439: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-12 17:52:14.276168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 17:52:14.302121: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 17:52:14.302173:

In [2]:
# 모델 및 토크나이저 초기화
model_name = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMultipleChoice.from_pretrained(model_name)

Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
train

,id,input,output
0,nikluge-2024-대화 맥락 추론-train-000001,"{'conversation': [{'speaker': 2, 'utterance': ...",inference_2
1,nikluge-2024-대화 맥락 추론-train-000002,"{'conversation': [{'speaker': 2, 'utterance': ...",inference_3
2,nikluge-2024-대화 맥락 추론-train-000003,"{'conversation': [{'speaker': 2, 'utterance': ...",inference_3
3,nikluge-2024-대화 맥락 추론-train-000004,"{'conversation': [{'speaker': 2, 'utterance': ...",inference_3
4,nikluge-2024-대화 맥락 추론-train-000005,"{'conversation': [{'speaker': 2, 'utterance': ...",inference_2
...,...,...,...
753,nikluge-2024-대화 맥락 추론-train-000754,"{'conversation': [{'speaker': 2, 'utterance': ...",inference_3
754,nikluge-2024-대화 맥락 추론-train-000755,"{'conversation': [{'speaker': 1, 'utterance': ...",inference_3
755,nikluge-2024-대화 맥락 추론-train-000756,"{'conversation': [{'speaker': 1, 'utterance': ...",inference_2
756,nikluge-2024-대화 맥락 추론-train-000757,"{'conversation': [{'speaker': 1, 'utterance': ...",inference_3


In [4]:
test

,id,input,output
0,nikluge-2024-대화 맥락 추론-test-000001,"{'conversation': [{'speaker': 1, 'utterance': ...",
1,nikluge-2024-대화 맥락 추론-test-000002,"{'conversation': [{'speaker': 1, 'utterance': ...",
2,nikluge-2024-대화 맥락 추론-test-000003,"{'conversation': [{'speaker': 1, 'utterance': ...",
3,nikluge-2024-대화 맥락 추론-test-000004,"{'conversation': [{'speaker': 1, 'utterance': ...",
4,nikluge-2024-대화 맥락 추론-test-000005,"{'conversation': [{'speaker': 1, 'utterance': ...",
...,...,...,...
600,nikluge-2024-대화 맥락 추론-test-000601,"{'conversation': [{'speaker': 1, 'utterance': ...",
601,nikluge-2024-대화 맥락 추론-test-000602,"{'conversation': [{'speaker': 2, 'utterance': ...",
602,nikluge-2024-대화 맥락 추론-test-000603,"{'conversation': [{'speaker': 2, 'utterance': ...",
603,nikluge-2024-대화 맥락 추론-test-000604,"{'conversation': [{'speaker': 2, 'utterance': ...",


### **2.데이터 전처리** 

In [5]:
mapping = {'inference_1': 0, 'inference_2': 1, 'inference_3': 2, '': -1}

train['output'] = train['output'].map(mapping)
dev['output'] = dev['output'].map(mapping)
#test['output'] = test['output'].map(mapping)

In [6]:
test

,id,input,output
0,nikluge-2024-대화 맥락 추론-test-000001,"{'conversation': [{'speaker': 1, 'utterance': ...",
1,nikluge-2024-대화 맥락 추론-test-000002,"{'conversation': [{'speaker': 1, 'utterance': ...",
2,nikluge-2024-대화 맥락 추론-test-000003,"{'conversation': [{'speaker': 1, 'utterance': ...",
3,nikluge-2024-대화 맥락 추론-test-000004,"{'conversation': [{'speaker': 1, 'utterance': ...",
4,nikluge-2024-대화 맥락 추론-test-000005,"{'conversation': [{'speaker': 1, 'utterance': ...",
...,...,...,...
600,nikluge-2024-대화 맥락 추론-test-000601,"{'conversation': [{'speaker': 1, 'utterance': ...",
601,nikluge-2024-대화 맥락 추론-test-000602,"{'conversation': [{'speaker': 2, 'utterance': ...",
602,nikluge-2024-대화 맥락 추론-test-000603,"{'conversation': [{'speaker': 2, 'utterance': ...",
603,nikluge-2024-대화 맥락 추론-test-000604,"{'conversation': [{'speaker': 2, 'utterance': ...",


In [7]:
train

,id,input,output
0,nikluge-2024-대화 맥락 추론-train-000001,"{'conversation': [{'speaker': 2, 'utterance': ...",1
1,nikluge-2024-대화 맥락 추론-train-000002,"{'conversation': [{'speaker': 2, 'utterance': ...",2
2,nikluge-2024-대화 맥락 추론-train-000003,"{'conversation': [{'speaker': 2, 'utterance': ...",2
3,nikluge-2024-대화 맥락 추론-train-000004,"{'conversation': [{'speaker': 2, 'utterance': ...",2
4,nikluge-2024-대화 맥락 추론-train-000005,"{'conversation': [{'speaker': 2, 'utterance': ...",1
...,...,...,...
753,nikluge-2024-대화 맥락 추론-train-000754,"{'conversation': [{'speaker': 2, 'utterance': ...",2
754,nikluge-2024-대화 맥락 추론-train-000755,"{'conversation': [{'speaker': 1, 'utterance': ...",2
755,nikluge-2024-대화 맥락 추론-train-000756,"{'conversation': [{'speaker': 1, 'utterance': ...",1
756,nikluge-2024-대화 맥락 추론-train-000757,"{'conversation': [{'speaker': 1, 'utterance': ...",2


In [8]:
# 대화 생성 함수
def make_chat(inp):
    chat = [""]
    current_speaker = 0
    for cvt in inp['conversation']:
        speaker = cvt['speaker']
        utterance = cvt['utterance']
        if current_speaker != speaker:
            chat.append(f"화자{speaker}: {utterance}")
            current_speaker = speaker
        else:
            chat.append(f"{utterance}")
    chat = "\n".join(chat)
    
    
    #question = f"[{inp['category']}]"
    question = inp['category']
    #inferences = [f"{i+1} - {inp[f'inference_{i+1}']}" for i in range(3)]
    A = inp['inference_1']
    B = inp['inference_2']
    C = inp['inference_3']
    #inference_candidates = f"{' '.join(inferences)}"
    #dchat = f"{inference_candidates}\n\n[SEP]{chat}"
    return clean_str(chat), A,B,C, question

# 텍스트 정제 함수
def clean_str(text):
    text = re.sub(r'http\S+|www\S+|https\S+|\S+@\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^\w\s\n\[\]\?\.,!]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [9]:
dev_preprocess = [make_chat(inp) for inp in dev['input']]
train_preprocess = [make_chat(inp) for inp in train['input']]
test_preprocess = [make_chat(inp) for inp in test['input']]

In [10]:
dev_preprocess

[('화자1 안녕하세요! ㅎㅎ 화자2 안녕하세요! 동호회활동하세요? 화자1 저는 차 마시는 거 좋아해서 차 좋아하는 사람들이랑 가끔 다회 참석해요 name2님은 동호회활동 참여하는 거 있으신가요? 화자2 오오 아니요ㅋㅋㅋ저는 없어요 새로운 사람 만나는 걸 즐기지는 않아서 아직 시도를 못 해봤어요 동호회! 화자1 아하ㅎㅎ 저는 직업상 만나는 사람들이 한정적이어서 동호회 활동을 하니까 다양한 사람을 만나게 되어 좋더라고요 저도 굉장히 내향적인 편인데 동호회 안에서 친해진 사람들하고만 만나고 있어요 화자2 이상하게 동호회는 목적이 다르다는 편견이 좀 있어서 망설여지더라고요! 물론 다 그런 게 아닌데! 화자1 아 무슨 말씀이신지 너무 알 것 같아요 불순한 목적으로 동호회 참석하는 사람들 때문에 좀 이미지가 그렇긴 하죠 특히 남자..들.. 이런 말 해도 되나요 ㅋㅋㅋ 화자2 으으 맞아요 맞아 안 좋은 얘기도 사실 많이 들어서... 화자1 제가 참석하는 차 동호회는 20대 여자가 대부분이라 더 편하게 만날 수 있는 것 같아요ㅠ 여성 동호회 쪽으로 알아보시면 좀 걱정이 덜하지 않을까요 화자2 오! 그런 방법이 있겠네요 그리고 어떤 분이 그러셨는데 약간 돈이 드는 동호회? 는 오히려 그런 사람이 적대요ㅋㅋㅋㅋㅋㅋ 화자1 돈이 드는 동호회 ㅋㅋㅋㅋㅋ 차는 돈이 좀 들어서 그런 걸까요 화자2 네! 그럴지도 몰라요 동호회는 인터넷에서 검색해서 들어가는 건가요? 화자1 저는 트위터로 시작해서 마음 맞는 사람들이랑 같이 만나고 있어요 화자2 오호 트위터! 그런 방법도 있네요 정말 트위터하면 아무래도 관심사가 비슷한 사람을 만나게 되니까요 화자1 맞아요 맞아요',
  '화자2는 미혼 남녀 대상의 동호회를 알아볼 것이다. ',
  '화자2는 가입 가능한 여성 동호회를 알아볼 것이다.',
  '화자2는 연애를 할 목적으로 동호회를 알아볼 것이다.',
  '후행사건'),
 ('화자1 안녕하세요, 어떤 공연을 주로 관람하시나요? 화자2 공연은 보고싶은데 기회가 많이 없어서.. 음 공연 자주 

In [11]:
import pandas as pd 


train_df = pd.DataFrame.from_dict(train_preprocess)
train_df.columns = ["conversation","inference_1","inference_2","inference_3","task_prompt"]
train_df.insert(5,'label',train['output'])

train_df

,conversation,inference_1,inference_2,inference_3,task_prompt,label
0,화자2 진짜 신의 한수 화자1 이사하자마자 비 많이 와서 베란다 물 많이 새는 거 ...,화자2가 사는 곳 근처에서 베란다 보수 공사가 진행되고 있다.,화자2가 사는 곳 근처에서 싱크홀 보수 공사가 진행되고 있다.,화자2가 사는 곳 근처에서 싱크홀 보수 공사가 중단되었다.,원인,1
1,화자2 진짜 신의 한수 화자1 이사하자마자 비 많이 와서 베란다 물 많이 새는 거 ...,화자2는 혹시 모를 위험을 방관하고 싶다.,화자2는 과거에 발생한 사건을 수습하고 싶다.,화자2는 혹시 모를 위험에 대비하고 싶다.,동기,2
2,화자2 진짜 신의 한수 화자1 이사하자마자 비 많이 와서 베란다 물 많이 새는 거 ...,화자2가 거주 중인 지역은 강수량이 적다.,화자2가 거주 중인 지역의 땅은 지반이 튼튼하다.,화자2가 거주 중인 지역의 땅은 지반이 약하다.,전제,2
3,화자2 진짜 신의 한수 화자1 이사하자마자 비 많이 와서 베란다 물 많이 새는 거 ...,화자1은 조심하려고 노력하는 화자2를 보니 초조하다.,화자1은 조심하려고 노력하는 화자2가 황당하다.,화자1은 조심하려고 노력하는 화자2가 바람직스럽다.,반응,2
4,화자2 진짜 신의 한수 화자1 이사하자마자 비 많이 와서 베란다 물 많이 새는 거 ...,화자2는 베란다 공사를 진행하기 위해 시공업체와 전화 통화를 할 것이다.,화자2는 싱크홀 공사가 진행 중인 곳을 피해 갈 수 있는 경로를 알아볼 것이다.,화자2는 싱크홀 공사가 진행 중인 곳을 방문할 계획을 세울 것이다.,후행사건,1
...,...,...,...,...,...,...
753,화자2 요즘은 동호회 안하니? 화자1 네 화자2 코로나라 그런거냐? 화자1 그런것도...,화자1은 아빠께 인라인스케이트를 선물해 드릴 것이다.,화자1은 자신의 인라인스케이트를 중고로 판매할 것이다.,화자1은 인라인스케이트를 탈 만할 곳을 찾아볼 것이다.,후행사건,2
754,화자1 밥은 먹엇니 화자2 ㅋㅋㅋ파스타먹엇죵 화자1 키키 잘했네 나는 김밥먹어뜌 오...,화자2는 미용실에 다녀왔다.,화자2는 최근 머리를 잘랐다.,화자2는 미용실을 예약했다.,원인,2
755,화자1 밥은 먹엇니 화자2 ㅋㅋㅋ파스타먹엇죵 화자1 키키 잘했네 나는 김밥먹어뜌 오...,화자2는 화자1과 서먹하다.,화자2는 화자1과 만나고 싶다.,화자2는 화자1의 머리를 잘라주고 싶다.,동기,1
756,화자1 밥은 먹엇니 화자2 ㅋㅋㅋ파스타먹엇죵 화자1 키키 잘했네 나는 김밥먹어뜌 오...,화자1은 화자2와의 만남이 꺼려진다.,화자1은 화자2와의 약속이 부담된다.,화자1은 화자2와의 만남이 기대된다.,반응,2


In [12]:
dev_df = pd.DataFrame.from_dict(dev_preprocess)
dev_df.columns = ["conversation","inference_1","inference_2","inference_3","task_prompt"]
dev_df.insert(5,'label',dev['output'])

dev_df

,conversation,inference_1,inference_2,inference_3,task_prompt,label
0,화자1 안녕하세요! ㅎㅎ 화자2 안녕하세요! 동호회활동하세요? 화자1 저는 차 마시...,화자2는 미혼 남녀 대상의 동호회를 알아볼 것이다.,화자2는 가입 가능한 여성 동호회를 알아볼 것이다.,화자2는 연애를 할 목적으로 동호회를 알아볼 것이다.,후행사건,1
1,"화자1 안녕하세요, 어떤 공연을 주로 관람하시나요? 화자2 공연은 보고싶은데 기회가...",화자2는 옥션에서 티켓팅을 할 때 생긴 일을 이야기할 것이다.,화자2는 옥션에서 항상 티켓팅에 성공할 수 있었던 비결을 알려줄 것이다.,화자2는 옥션의 서버가 특히 안정적인 이유를 알려줄 것이다.,후행사건,0
2,화자1 3기 신도시에 관해서 아시는거 있으세요? 화자2 헉 이런 주제가 화자1 정말...,화자2는 자신이 신청할 수 있는 유형의 청약을 알아볼 것이다.,화자2는 신혼부부가 신청할 수 있는 청약 유형을 검색할 것이다.,화자2는 청약에 당첨되기 위해 결혼 날짜를 앞당길 것이다.,후행사건,0
3,화자2 요즘 보시는 티비프로 같은거 있으신가요? 화자1 놀면뭐하지 봐요! 주말드라마...,화자2는 '편스토랑' 시청을 중단할 것이다.,화자2는 '편스토랑' 채널을 차단할 것이다.,화자2는 '편스토랑'을 시청할 것이다.,후행사건,2
4,화자2 가사노동ㅠ 화자1 ㅠㅠㅠ 해도해도 끝이 없는 집안일.. 화자2 마자요 돈도안...,화자1은 자신의 로봇청소기를 중고로 팔 것이다.,화자1은 로봇청소기로 청소를 시작할 것이다.,화자1은 로봇청소기 최저가를 검색할 것이다.,후행사건,2
...,...,...,...,...,...,...
146,화자2 버뜨 건성은 겨울에 화장이 굳어서 쩍쩍 갈라진다는 거ㅋㅋㅋㅋ 화자1 수분 부...,화자2는 피부가 건성으로 바뀌었으면 싶다.,화자2는 건성인 피부를 개선하고 싶다.,화자2는 건성인 피부를 유지하고 싶다.,동기,1
147,화자2 버뜨 건성은 겨울에 화장이 굳어서 쩍쩍 갈라진다는 거ㅋㅋㅋㅋ 화자1 수분 부...,화자2는 피부과에서 일한다.,화자2는 피부 관리를 한다.,화자2는 피부에 기름이 많다.,전제,1
148,화자2 버뜨 건성은 겨울에 화장이 굳어서 쩍쩍 갈라진다는 거ㅋㅋㅋㅋ 화자1 수분 부...,화자1은 자신과 달리 건성 피부에 만족하는 화자2가 신기하다.,화자1은 자신처럼 피부가 건조해 괴로워하는 화자2가 반갑다.,화자1은 자신과 달리 건성 피부로 힘들어하는 화자2가 신기하다.,반응,2
149,화자2 버뜨 건성은 겨울에 화장이 굳어서 쩍쩍 갈라진다는 거ㅋㅋㅋㅋ 화자1 수분 부...,화자2는 건성 피부 유형에 좋은 화장품을 찾아볼 것이다.,화자2는 지성 피부에 좋은 화장품을 알아볼 것이다.,화자2는 지성 피부를 잘 챙겨준다는 피부과를 알아볼 것이다.,후행사건,0


In [13]:
test_df = pd.DataFrame.from_dict(test_preprocess)
test_df.columns = ["conversation","inference_1","inference_2","inference_3","task_prompt"]
#test_df.insert(5,'label',test['output'])

test_df

,conversation,inference_1,inference_2,inference_3,task_prompt
0,화자1 복수전공이면 엄청 힘들었을 것 같은데요 화자2 엄마 집에 있으니까 단점이 있...,화자2의 자취 집에 있는 거실 화장실은 방음이 잘 안된다.,화자2의 어머님 집 거실에 있는 화장실은 방음이 잘 안된다.,화자2의 어머님 집 거실에 있는 화장실은 방음이 잘된다.,원인
1,화자1 복수전공이면 엄청 힘들었을 것 같은데요 화자2 엄마 집에 있으니까 단점이 있...,화자2는 청결하고 넓은 화장실에서 볼일을 보고 싶다.,화자2는 음악을 들으며 화장실을 이용하고 싶다.,화자2는 소리를 걱정하지 않고 화장실을 이용하고 싶다.,동기
2,화자1 복수전공이면 엄청 힘들었을 것 같은데요 화자2 엄마 집에 있으니까 단점이 있...,화자2는 혼자 독립하여 살고 있다.,화자2는 현재 어머님 집에 살고 있다.,화자2는 한 번도 자취한 적이 없다.,전제
3,화자1 복수전공이면 엄청 힘들었을 것 같은데요 화자2 엄마 집에 있으니까 단점이 있...,화자1은 평범한 화자2의 습관에 무관심하다.,화자1은 특이한 화자2의 습관이 섬뜩하다.,화자1은 특이한 화자2의 습관이 신기하다.,반응
4,화자1 복수전공이면 엄청 힘들었을 것 같은데요 화자2 엄마 집에 있으니까 단점이 있...,화자2는 카페 화장실의 장점을 이야기할 것이다.,화자2는 어머님 집 화장실의 장점을 이야기할 것이다.,화자2는 자신이 최근 즐겨듣는 음악을 공유할 것이다.,후행사건
...,...,...,...,...,...
600,화자1 잘 할 줄 모르면 밀키트 좋은 거 같아요 화자2 오 짱좋아요 ㅋㅋㅋ 아 궁금...,화자1은 새로운 직장을 찾아볼 것이다.,화자1은 현 직장을 그만둘 것이다.,화자1은 직장에서 승진을 준비할 것이다.,후행사건
601,화자2 나는 산보다는 바다가 좋아. 나는 부산에 또 가는 것도 좋고 강릉도 좋아 아...,화자1은 대만의 비행기 편 가격을 찾아볼 것이다.,화자1은 대만 여행에서 느낀 점을 공유할 것이다.,화자2는 중국어 학원에 등록 문의 전화를 할 것이다.,후행사건
602,화자2 음....중고폰 할부...? 화자1 헐 폰바꿈? 화자2 너 만났을때 이미 바...,화자1은 예전에 하나의 폴더폰을 얼마나 오래 사용했는지 말할 것이다.,화자1은 고장이 나지 않는 요즘 휴대폰에 적용된 신기술을 알아볼 것이다.,화자1은 예전에 폴더폰이 금방 고장 나 생긴 일화를 공유할 것이다.,후행사건
603,화자2 전시도 좋아하세요? 화자1 전시도 좋아하는데 그것도 19년도 겨울이 마지막이...,화자1은 마크 로스코의 그림을 담은 엽서를 구입할 것이다.,화자1은 마크 로스코 전에서 찍은 사진을 볼 것이다.,화자1은 마크 로스코 전시 일정을 찾아볼 것이다.,후행사건


### **3.탐색적데이터 분석 (EDA)**

In [14]:
cnts = train_df['task_prompt'].value_counts()
percs = train_df['task_prompt'].value_counts(normalize=True)

print(pd.concat([cnts, percs], axis=1, keys=['count','percentage']))

             count  percentage
task_prompt                   
반응             162    0.213720
원인             161    0.212401
전제             161    0.212401
동기             156    0.205805
후행사건           118    0.155673


In [15]:
cnts = dev_df['task_prompt'].value_counts()
percs = dev_df['task_prompt'].value_counts(normalize=True)

print(pd.concat([cnts, percs], axis=1, keys=['count','percentage']))

             count  percentage
task_prompt                   
후행사건            51    0.337748
원인              25    0.165563
동기              25    0.165563
전제              25    0.165563
반응              25    0.165563


In [16]:
cnts = test_df['task_prompt'].value_counts()
percs = test_df['task_prompt'].value_counts(normalize=True)

print(pd.concat([cnts, percs], axis=1, keys=['count','percentage']))

             count  percentage
task_prompt                   
후행사건           133    0.219835
동기             124    0.204959
원인             117    0.193388
반응             116    0.191736
전제             115    0.190083


In [17]:
cnts = train_df['label'].value_counts()
percs = train_df['label'].value_counts(normalize=True)

print(pd.concat([cnts, percs], axis=1, keys=['count','percentage']))

       count  percentage
label                   
2        262    0.345646
0        260    0.343008
1        236    0.311346


In [18]:
cnts = dev_df['label'].value_counts()
percs = dev_df['label'].value_counts(normalize=True)

print(pd.concat([cnts, percs], axis=1, keys=['count','percentage']))

       count  percentage
label                   
2         53    0.350993
1         50    0.331126
0         48    0.317881


In [ ]:
#cnts = test_df['label'].value_counts()
#percs = test_df['label'].value_counts(normalize=True)

#print(pd.concat([cnts, percs], axis=1, keys=['count','percentage']))

In [ ]:
train_tokens = train_df['conversation'].apply(lambda s: tokenizer.tokenize(s))
train_token_lengths = train_df['conversation'].apply(lambda s: len(tokenizer.tokenize(s)))

dev_tokens = dev_df['conversation'].apply(lambda s: tokenizer.tokenize(s))
dev_token_lengths = dev_df['conversation'].apply(lambda s: len(tokenizer.tokenize(s)))

test_tokens = test_df['conversation'].apply(lambda s: tokenizer.tokenize(s))
test_token_lengths = test_df['conversation'].apply(lambda s: len(tokenizer.tokenize(s)))

In [ ]:


train_cnt, dev_cnt, test_cnt = 0, 0, 0 
for i in train_token_lengths:
    if i > 512: train_cnt += 1
for i in dev_token_lengths:
    if i > 512: dev_cnt += 1
for i in test_token_lengths:
    if i > 512: test_cnt += 1        


In [ ]:
print(f"평균 토큰 수 : {train_token_lengths.mean()}")
print(f"최대 토큰 수 : {train_token_lengths.max()}")
print(f"최소 토큰 수 : {train_token_lengths.min()}")
print(f"토큰 수 512개 초과 : {train_cnt}")

In [ ]:
print(f"평균 토큰 수 : {dev_token_lengths.mean()}")
print(f"최대 토큰 수 : {dev_token_lengths.max()}")
print(f"최소 토큰 수 : {dev_token_lengths.min()}")
print(f"토큰 수 512개 초과 : {dev_cnt}")

In [ ]:
print(f"평균 토큰 수 : {test_token_lengths.mean()}")
print(f"최대 토큰 수 : {test_token_lengths.max()}")
print(f"최소 토큰 수 : {test_token_lengths.min()}")
print(f"토큰 수 512개 초과 : {test_cnt}")

### **4.모델 학습**

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
choice_names = ["inference_1", "inference_2", "inference_3"]

def preprocess_function(examples):
    first_sentences = [[context] * 3 for context in examples["conversation"]]
    question_headers = examples["task_prompt"]
    second_sentences = [
        [f"{header} {examples[end][i]}" for end in choice_names] for i, header in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + 3] for i in range(0, len(v), 3)] for k, v in tokenized_examples.items()}

In [ ]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_train_dataset

In [ ]:
tokenized_dev_dataset = dev_dataset.map(preprocess_function, batched=True)
tokenized_dev_dataset

In [ ]:
from datasets import concatenate_datasets

concat_dataset = concatenate_datasets([tokenized_train_dataset, tokenized_dev_dataset])
concat_dataset = concat_dataset.train_test_split(test_size=0.2)

In [ ]:
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
accepted_keys = ['input_ids','attention_mask','label'] 

features = [{k: v for k, v in tokenized_test_dataset[i].items() if k in accepted_keys} for i in range(len(tokenized_test_dataset))]

features

In [ ]:
accepted_keys = ['input_ids','attention_mask','label'] 

features = [{k: v for k, v in tokenized_train_dataset[i].items() if k in accepted_keys} for i in range(len(tokenized_train_dataset))]

features

In [ ]:
batch = DataCollatorForMultipleChoice(tokenizer)(features)

In [ ]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(3)]

In [ ]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
concat_dataset['train']

In [ ]:
import torch

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

# 모델 및 데이터 GPU에 할당
model.to(device)

# 데이터 로더에서 배치 크기 줄이기
training_args = TrainingArguments(
    output_dir="my_awesome_ondam_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=0.00001,
    per_device_train_batch_size=8,  # 줄인 배치 크기
    per_device_eval_batch_size=16,   # 줄인 배치 크기
    num_train_epochs=10,
    weight_decay=0.001,
    #push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    #train_dataset=tokenized_train_dataset,
    #eval_dataset=tokenized_dev_dataset,
    train_dataset=concat_dataset['train'],
    eval_dataset=concat_dataset['test'],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()



### **5. 테스트 데이터 검증**

In [ ]:

# 레이블 없이 테스트 데이터 예측
#test_features = test_dataset.remove_columns(["label"])
tokenized_test_features = test_dataset.map(preprocess_function, batched=True)



In [ ]:
test_predictions = trainer.predict(tokenized_test_features)